# Feedback Prize - Evaluating Student Writing
#### Analyze argumentative writing elements from students grade 6-12
![](https://storage.googleapis.com/kaggle-competitions/kaggle/31779/logos/header.png?t=2021-11-12-22-52-17&quot)

https://drive.google.com/file/d/1r9vxKn5Az3ZBoZ7PgkmIh2ov87axyN8B/view

# Import packages

In [ ]:
# Install packages

In [ ]:
# Import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from tqdm import tqdm

# Load data

In [ ]:
train_data = pd.read_csv('../input/feedback-prize-2021/train.csv')
submission_data = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

# EDA

In [ ]:
print(train_data.shape)
train_data.head()

In [ ]:
print('Discourse types in train:\n', len(train_data['discourse_type'].unique().tolist()), 'types -', train_data['discourse_type'].unique().tolist())

In [ ]:
print('# Splits for first passage:', train_data[train_data['id']==train_data['id'].iloc[0]]['discourse_id'].nunique())
train_data[train_data['id']==train_data['id'].iloc[0]]

In [ ]:
print(submission_data.shape)
submission_data.head()

## Check prediction string

In [ ]:
''.join(train_data[train_data['id']==train_data['id'].unique()[2]]['predictionstring'].tolist())

In [ ]:
len(' '.join(train_data[train_data['id']==train_data['id'].unique()[2]]['discourse_text'].apply(lambda x: re.sub('\n+',' ',x)).str.strip().tolist()).split(' ')), len(' '.join(train_data[train_data['id']==train_data['id'].unique()[2]]['predictionstring'].tolist()).split(' '))

In [ ]:
filepath = '../input/feedback-prize-2021/'
record_id = 'A97DE0D49AEA'
# record_id = '1CEDD5563788'
with open(filepath + 'train/' + record_id+'.txt','r') as f:
    data = f.read()

In [ ]:
train_data[train_data['id']==record_id]

In [ ]:
re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')

Combined words create an issue e.g: "DrivingHundreds" in the above passage

Also references are not to be tagged under the 7 classes

In [ ]:
re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0].strip().replace("\xa0"," ").replace('"',''))

In [ ]:
print('Prediction string in train data should start at:', len(re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')[0:re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','').find(' '.join(re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0].strip().replace("\xa0"," ").replace('"','')).split(' ')[0:4]))].strip().split(' ')))

In [ ]:
len(re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0]).strip().replace("\xa0"," ").replace('"','').split(' ')), re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0]).strip().replace("\xa0"," ").replace('"','').split(' ')

In [ ]:
len(train_data[train_data['id']==record_id]['predictionstring'].iloc[0].split(' ')), train_data[train_data['id']==record_id]['predictionstring'].iloc[0].split(' ')

In [ ]:
train_data['cleaned_discourse_text'] = train_data['discourse_text'].apply(lambda x: re.sub('\n+',' ',x).strip().replace("\xa0"," ").replace('"',''))
train_data['Flag - pred len and text len'] = np.where(len(train_data['cleaned_discourse_text'].str.split(' '))==len(train_data['predictionstring'].str.split(' ')), 1, 0)

mismatches_pred_string = []
records_with_name_mask = []
record_id_list = []

for i in tqdm(range(0, train_data.shape[0])):
    record_id = train_data['id'].iloc[i]
    if record_id not in record_id_list:
        with open(filepath + 'train/' + record_id+'.txt','r') as f:
            data = f.read()

        data = re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')
        str_index = data.find(train_data[train_data['id']==record_id]['cleaned_discourse_text'].iloc[0])
        words_before_pred_string_start = data[:str_index].strip().split(' ')

        if str_index != -1:
            if (len(words_before_pred_string_start) != int(train_data['predictionstring'].iloc[i].strip().split(' ')[0])) and (words_before_pred_string_start[0] != ''):
                mismatches_pred_string.append(record_id)
                if ('PROPER_NAME' in data) or ('GENERIC_NAME' in data):
                    records_with_name_mask.append(record_id)
        else:   
            # At least first 4 words are correct
            if (len(data[:data.find(' '.join(train_data[train_data['id']==record_id]['cleaned_discourse_text'].iloc[0].split(' ')[0:4]))].strip().split(' ')) != int(train_data['predictionstring'].iloc[i].strip().split(' ')[0])) and (data[:data.find(' '.join(train_data[train_data['id']==record_id]['cleaned_discourse_text'].iloc[0].split(' ')[0:4]))].strip().split(' ')[0] != ''):
                mismatches_pred_string.append(record_id)
                if ('PROPER_NAME' in data) or ('GENERIC_NAME' in data):
                    records_with_name_mask.append(record_id)
        record_id_list.append(record_id)

In [ ]:
print('# Records with number of words in discourse text not equal to number of indices in prediction string:', train_data[train_data['Flag - pred len and text len'] == 0].shape[0])

In [ ]:
print('# Misatches:', len(mismatches_pred_string), 'out of', train_data['id'].nunique(), 'ids in train data', '(' + str(len(mismatches_pred_string)/train_data['id'].nunique() * 100) + '%)')

In [ ]:
mismatches_pred_string

In [ ]:
len([x for x in mismatches_pred_string if x not in records_with_name_mask]), [x for x in mismatches_pred_string if x not in records_with_name_mask]

In [ ]:
# # Check if ending of predictionstring matches with beginning of next predictionstring
# start_not_matching_end_discourse_ids = []
# for i in tqdm(range(0, train_data.shape[0])):
#     temp = train_data[(train_data['id']==train_data['id'].iloc[i])].reset_index(drop = True)
#     temp_index = temp[temp['discourse_id']==train_data['discourse_id'].iloc[i]].index[0]
#     if temp_index > 0:
#         if (temp['discourse_id'].iloc[temp_index-1] != train_data['discourse_id'].iloc[i]):
#             previous_pred = temp['predictionstring'].iloc[temp_index-1].strip().split(' ')[-1]
#             start_pred = train_data['predictionstring'].iloc[i].strip().split(' ')[0]
#             if int(start_pred) != int(previous_pred) + 1:
#                 start_not_matching_end_discourse_ids.append(train_data['discourse_id'].iloc[i])

In [ ]:
record_id = 'E05C7F5C1156'

with open(filepath + 'train/' + record_id+'.txt','r') as f:
    data = f.read()

In [ ]:
train_data[train_data['id']==record_id]

In [ ]:
re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')

In [ ]:
re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[4].strip().replace("\xa0"," ").replace('"',''))

In [ ]:
re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[5].strip().replace("\xa0"," ").replace('"',''))

The sentence "Therefore, driving can cause many accidents that can be fatal to the driver and passengers if there is any and cell phones should only be used when not operating a vehicle." does not have any tagged class

# WIP